In [1]:
import pandas as pd
import numpy  as np
import time
import requests as rq
import json
import geohash
import TransCooSys
# 根目录
ROOT_DIR = 'D:/py_workspace/'
ADDR_DIR = ROOT_DIR+'addr_/'
# 资源目录
RESOURECE_DIR = ROOT_DIR + 'resources/'

In [3]:
df1 = pd.read_csv(ROOT_DIR+'crawl_input/2022年全国行政区域.txt',encoding='utf-8', header=None, sep='\t')
df2 = pd.read_csv(RESOURECE_DIR+'all_street_0526.txt',encoding='utf-8', header=None, sep='\t')
s1 = set(np.array(df1[7]).tolist())
#s1 = set(np.array(df1[0]).tolist())
s2 = set(np.array(df2[6]).tolist())


In [2]:
df = pd.read_csv(ADDR_DIR+'address_hangzhou_new.txt', names=['addr'])
addr_list = df['addr'].to_list()
addr_list

['浙江省杭州市临平区塘栖镇兴元路杭辰星万里7幢2单元1602单元室',
 '浙江省杭州市上城区解放路1号泊寓公寓丰巢',
 '浙江省杭州市临平区崇贤街道三家村舍郎105号',
 '浙江省杭州市临安区青山湖街道湖畔家园绎站',
 '浙江省杭州市临平区临平街道临平街道绿城蓝庭蔷薇院3-2-402',
 '浙江省杭州市余杭区桃源小镇芳华苑11-1-1201',
 '浙江省杭州市上城区小营街道佑圣观路51号',
 '浙江省杭州市上城区九堡街道 金堡街7-2[3873]',
 '浙江省杭州市上城区紫阳街道金狮苑6-1 -702',
 '浙江省杭州市临平区塘栖镇柴家坞村西南墩桥145号',
 '浙江省杭州市余杭区良渚街道金家渡铭雅苑东区49幢二单元311',
 '浙江省杭州市富阳区银湖街道浙大网新银湖科技园27幢放一楼大厅',
 '浙江省杭州市余杭区径山镇国光毛巾厂俞顺路8号',
 '浙江省杭州市余杭区良渚街道良和雅苑 5幢',
 '浙江省杭州市富阳区富春街道东山村东山三路118号102室',
 '浙江省杭州市建德市李家镇沙墩头村北坑源5号',
 '浙江省杭州市余杭区文一西路常二路交汇乐淘城1-3-1101',
 '浙江省杭州市余杭区余杭街道杭州市余杭区桃源小镇东方苑7幢1503室',
 '浙江省杭州市上城区彭埠街道红建街红五月嘉苑2区7幢1单元1101',
 '浙江省杭州市余杭区东安景苑8栋2单元901',
 '浙江省杭州市上城区滨江金色家园8幢3单元701',
 '浙江省杭州市余杭区五常街道合景映月台5幢3单元502',
 '浙江省杭州市临平区乔莫东路92号杭州李冰客房',
 '浙江省杭州市富阳区灵桥王家宕路86号金富春公寓旁小吃店',
 '浙江省杭州市临安区青山湖街道大园新城北区9幢1单元802室',
 '浙江省杭州市富阳区店口家亭桥头村21号',
 '浙江省杭州市临安区相坞183号快递超市',
 '浙江省杭州市富阳区富春街道春秋南路13-1',
 '浙江省杭州市余杭区良渚镇七贤桥村七贤山居86号',
 '浙江省杭州市余杭区黄湖镇黄湖镇虎山村毛家17号（虎山村村委旁）',
 '浙江省杭州市富阳区浙江省杭卅市富阳区鹿山街道南山中埠变电所',
 '浙江省杭州市上城区四季青街道四季青街道钱江新城解放东路45号高德置地4单元26楼',
 '浙江省杭州市余杭区乔司街道和睦桥村五组

In [3]:
def get_json_http(url):
    try:
        r = rq.get(url, timeout=(3, 9))
        json_data = json.loads(r.text)
        return json_data
    except Exception as e:
        time.sleep(3)
        return get_json_http(url)
        
def get_baidu_content(keywords, c='268'):
    url = 'https://map.baidu.com/?newmap=1&qt=s&da_src=searchBox.button&wd={}&c={}'.format(keywords, c)
    json_id_data = get_json_http(url)
    return json_id_data
        

# 使用百度爬去geohash

In [21]:
def process_poi_geohash(keywords):
    try:
        content = get_baidu_content(keywords)['content']
        content0 = content[0]
        px = float(content0['diPointX'])/100
        py = float(content0['diPointY'])/100
        name = content0['name']
        addr = content0['addr']
    except Exception as e:
        return [keywords, '', '', '', '']
    coor_bd09 = TransCooSys.mercatortobd09(px, py)
    coor_wgs = TransCooSys.bd09_to_wgs84(coor_bd09[0], coor_bd09[1])
    coor_wgs = [round(coor_wgs[0],6), round(coor_wgs[1],6)]
    geo_str = geohash.encode(coor_wgs[1], coor_wgs[0], 6)
    return [keywords, name, addr, ','.join([str(i) for i in coor_wgs]), geo_str]

In [23]:
process_poi_geohash('浙江省杭州市建德市振兴路14号')


['浙江省杭州市建德市振兴路14号',
 '来一锅(振兴路店)',
 '浙江省杭州市建德市振兴路14号',
 '119.569251,29.538377',
 'wtm00c']

In [93]:
reslist = []
for i in range(0, len(addr_list)):
    address = addr_list[i]
    #poi = addr_list[i].split(';')[1]
    reslist.append(process_poi_geohash(address))
    if i%100 == 0:
        print(i)
reslist

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500


# 使用高德API爬取geohash

In [2]:
import requests
 
# 使用高德API
def geocodeG(address, key):
    par = {'address': address, 'key': key}
    base = 'http://restapi.amap.com/v3/geocode/geo'
    try:
        response = requests.get(base, par)
        answer = response.json()
        GPS = answer['geocodes'][0]['location'].split(",")
        formatted_address = answer['geocodes'][0]['formatted_address']
    except Exception as e:
        return [address, '', '', '']
    coor_wgs = TransCooSys.gcj02_to_wgs84(float(GPS[0]), float(GPS[1]))
    coor_wgs = [round(coor_wgs[0],6), round(coor_wgs[1],6)]
    geo_str = geohash.encode(coor_wgs[1], coor_wgs[0], 6)
    return [address, formatted_address, ','.join([str(i) for i in coor_wgs]), geo_str]

def poiGaode(address, key):
    par = {'address': address, 'key': key}
    base = 'http://restapi.amap.com/v3/geocode/geo'
    try:
        response = requests.get(base, par)
        answer = response.json()
        GPS = answer['geocodes'][0]['location'].split(",")
        formatted_address = answer['geocodes'][0]['formatted_address']
    except Exception as e:
        return [address, '', '', '']
    coor_wgs = TransCooSys.gcj02_to_wgs84(float(GPS[0]), float(GPS[1]))
    coor_wgs = [round(coor_wgs[0],6), round(coor_wgs[1],6)]
    geo_str = geohash.encode(coor_wgs[1], coor_wgs[0], 6)
    return [address, formatted_address, ','.join([str(i) for i in coor_wgs]), geo_str]

In [6]:
## 提取新的杭州目标地址
df = pd.read_csv(ADDR_DIR+'address_test.txt', index_col=None)
df=df.drop('cnee_unit',axis=1)
dlist = df.values.tolist()
newlist = []
area_list = ['上城区','拱墅区','西湖区','滨江区','萧山区','余杭区','富阳区','临安区','临平区','钱塘区','桐庐县','淳安县','建德市']
for line in dlist:
    if line[2] not in area_list:
        continue
    if '【' in line[3] or '？'in line[3] or '，'in line[3] or ':' in line[3] or len(line[3])<=5:
        continue
    newlist.append(''.join(line))
pd.DataFrame(newlist).sample(n=10000).to_csv('address_hangzhou_new_0906.txt', sep='\t', index=None, header=None)

In [7]:
df_addr = pd.read_csv(ADDR_DIR+'address_hangzhou_new_0906.txt', index_col=None)
addr_list = df_addr.values.tolist()
addr_list

[['浙江省杭州市富阳区富春街道金苑路308号 水岸青云小区碧水苑23幢202室'],
 ['浙江省杭州市上城区湖滨街道 定安路49号2单元504室'],
 ['浙江省杭州市余杭区云睦街闲林山水南门-2期丹枫苑15栋一单元902室'],
 ['浙江省杭州市临平区崇贤街道 崇贤群贤府26-1-302'],
 ['浙江省杭州市临平区南苑街道临平桂花城银桂苑13-1-601'],
 ['浙江省杭州市建德市寿昌镇建德市寿昌镇佳和公 寓'],
 ['浙江省杭州市余杭区棕榈湾二期红漾居6幢1单元101'],
 ['浙江省杭州市上城区笕桥街道全冯弄19号水墩苑2区3幢2单元602放门口'],
 ['浙江省杭州市富阳区东桥路16 9号水榭山居'],
 ['浙江省杭州市富阳区富春街道中光花园紫竹苑6号楼302'],
 ['浙江省杭州市余杭区西溪北苑西区61幢二单元401-8'],
 ['浙江省杭州市余杭区瓶窑镇彭公村两水坞组47-1号'],
 ['浙江省杭州市临平区南苑街道华元欢乐城东区尚峰轩 3幢2单元1404号'],
 ['浙江省杭州市富阳区新登镇新登镇新兴路230号（森强装饰）'],
 ['浙江省杭州市余杭区五常街道 西溪水岸花苑21幢'],
 ['浙江省杭州市临安区锦北街道金马村竹园里五组6号'],
 ['浙江省杭州市余杭区良渚街 道古墩路与金汇南路交汇处 上实海上海白洋港城1期 17幢1004'],
 ['浙江省杭州市上城区丁兰街道丁兰路7号华侨城芳菲与城14幢3单元301'],
 ['浙江省杭州市富阳区东洲街道东洲街道东洲工业功能区五号路慧颖超市'],
 ['浙江省杭州市临安区锦城街道城中街663号娃哈哈奶茶练刚'],
 ['浙江省杭州市余杭区仓前街道美瑭广场11-17-2喜姐炸串'],
 ['浙江省杭州市上城区四季青新中洲2058涵衣社'],
 ['浙江省杭州市上城区四季青街道钱江新城新业路8号UDC时代大厦B座1402 中国民生银行信用卡中心'],
 ['浙江省杭州市余杭区崇贤街道佳源 名城7-1-2604'],
 ['浙江省杭州市上城区四季青街道新业路8UDC时代大厦A幢楼25层'],
 ['浙江省杭州市上城区浙江省/杭州市/上城区/望江街道华龙坊10幢二单元202室'],
 ['浙江省杭州市余杭区良渚街道古墩路杜甫新苑杜康苑20幢1单元602'],
 ['浙江省杭

In [8]:
keys = [
    'fff94d523e87952085f10c62d183d865',
    'ac23bcb00bf5ae622c25666435434203',
    'bedc3209edf0db1c97c31a2a332807f2'
] 
k = 0
crawl_list = []
for i in range(0, len(addr_list)):
    if i%4000== 0:
        key = keys[k]
        k=k+1
        print(key)
    address = addr_list[i]
    crawl_list.append(geocodeG(address, key))
    if i%100 == 0:
        print(i)
    if i == 10000:
        break


fff94d523e87952085f10c62d183d865
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
ac23bcb00bf5ae622c25666435434203
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
bedc3209edf0db1c97c31a2a332807f2
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [5]:
address = '浙江省杭州市建德市振兴路14号'
key = 'fff94d523e87952085f10c62d183d865'
geocodeG(address, key)

# par = {'address': address, 'key': key}
# base = 'http://restapi.amap.com/v3/geocode/geo'

# response = requests.get(base, par)
# answer = response.json()
# answer

['浙江省杭州市建德市振兴路14号', '浙江省杭州市建德市振兴路|14', '120.520221,30.162614', 'wtmewd']

In [34]:
pd.DataFrame(crawl_list).to_csv('address_hangzhou_new_0906.txt', sep='\t')

In [130]:
def get_baidu_poi(keywords):
    try:
        content = get_baidu_content(keywords, '13')['content']
    except Exception as e:
        print(keywords)
        return None
    for content0 in content:
        try:
            name = content0['name']
            addr = content0['addr']
            std_tag = content0['std_tag']
            admin_info = content0['admin_info']
            area_id = str(admin_info['area_id'])
            town_id = str(admin_info['town_id'])+'000'
            street = admin_info['province_name']+';'+admin_info['city_name']+';'+admin_info['area_name']+';'+admin_info['town_name']
        except Exception as e:
            continue
        px = float(content0['diPointX'])/100
        py = float(content0['diPointY'])/100
        coor_bd09 = TransCooSys.mercatortobd09(px, py)
        coor_wgs = TransCooSys.bd09_to_wgs84(coor_bd09[0], coor_bd09[1])
        coor = str(round(coor_wgs[1],6))+' '+str(round(coor_wgs[0],6))
        break
    try:
        cpoi = '\t'.join([name, addr, area_id, std_tag, coor, town_id, street])
        return cpoi
    except Exception as e:
        return None

In [131]:
poilist = []
for res in reslist:
    cpoi = get_baidu_poi(res)
    if cpoi:
        poilist.append(cpoi)
poilist

西藏自治区山南市隆子县山南市隆子县出发路185号联系不上就帮忙拒签下
西藏自治区昌都市卡若区西藏自治区昌都市卡若区卧龙街西藏银行
西藏自治区拉萨市当雄县当雄县公路段对面。小尚车行，力帆摩托
西藏自治区拉萨市堆龙德庆区东嘎街道滨河路水亿方汤泉酒店
西藏自治区日喀则市仁布县德吉林镇仁布县新城区风情商业街对面刘妞木炭烧烤
西藏自治区日喀则市桑珠孜区城南街道   联系客服联系客服联系客服联系客服联系客服春城园西门
西藏自治区拉萨市堆龙德庆区柳语街道柳语乡海亮圆通1号转22号信箱
西藏自治区拉萨市城关区公德林街道罗堆东路11号92号信箱
西藏自治区昌都市察雅县烟多镇诸葛烤鱼店
西藏自治区林芝市波密县扎木镇扎木路昌运花园对面锦迅家具城收
西藏自治区拉萨市堆龙德庆区索南次旦电话号码18898006437所在地区:西藏自治区拉萨市堆龙德庆区详细地址:拉萨市堆龙德庆县古荣乡搬迁点
西藏自治区昌都市洛隆县孜托镇幸福小区1栋4单元（周转房）[8504]
西藏自治区那曲市比如县比如镇比如县第二小学对面单元楼
西藏自治区林芝市米林县派镇西藏林芝市米林县派镇中国农业银行
西藏自治区昌都市芒康县嘎托镇西藏昌都市芒康县圆通快递
西藏自治区拉萨市城关区夺底街道收货人:姐儿手机号码:18289190392所在地区:西藏自治区 拉萨市 城关区详细地址:夺底路西藏鸿宇消防工程有限公司楼下！马青龙藏式家具店
西藏自治区昌都市卡若区昌都市第三高级中学对面天天向上。
西藏自治区拉萨市其它区季华西路998号（多仓库发货，退货前务必联系客服，否则拒收）
西藏自治区昌都市贡觉县莫洛镇同美路27号香佰里私房菜
西藏自治区拉萨市城关区公德林街，当热西路八号政府二号院放在门口的架子上
西藏自治区山南市错那县公安局后面监狱16号信箱
西藏自治区日喀则市江孜县江热乡卫生院对面香缇小厨
西藏自治区日喀则市桑珠孜区体育路天临酒店旁边星辰茶馆
西藏自治区日喀则市桑珠孜区雪强支路进来150米左右的羊毛加工厂
西藏自治区拉萨市城关区拉萨市城关区功德林街道嘎吉路百惠缘超市对面巷子米欧公寓
西藏自治区日喀则市江孜县英雄南路，楚古宾馆对面潇雄日化批发
西藏自治区那曲市色尼区环城南路森林消防大队对面永康超市
西藏自治区拉萨市城关区公德林街道嘎吉路福利印刷厂西门对面巷子秋生百货30米星光公寓15号
西藏自治区日喀则市桑珠孜区城北街道雪强路夏林30

['西藏组织部\t拉萨市城关区娘热路5号\t540102\t政府机构;其他\t29.657753 91.121815\t540102004000\t西藏自治区;拉萨市;城关区;吉崩岗街道',
 '日喀则市人力资源和社区保障局\t西藏自治区日喀则市桑珠孜区黑龙江路22号\t540202\t政府机构;行政单位\t29.269525 88.888788\t540202001000\t西藏自治区;日喀则市;桑珠孜区;城南街道',
 '香皖吴记饭馆(比如店)\t比如镇南岸新区第二公务员小区五号门面\t540622\t美食;中餐厅\t31.478689 93.696806\t540622100000\t西藏自治区;那曲市;比如县;比如镇',
 '中共公德林街道委员会\t拉萨市城关区巴尔库路西藏自治区实验幼儿园北侧约140米\t540102\t政府机构;居民委员会\t29.667734 91.115867\t540102006000\t西藏自治区;拉萨市;城关区;公德林街道',
 '甲竹林镇\t西藏自治区山南市贡嘎县\t540522\t行政地标;乡镇\t29.286228 90.89339\t540522101000\t西藏自治区;山南市;贡嘎县;甲竹林镇',
 '羊达村\t拉萨市堆龙德庆区\t540103\t行政地标;村庄\t29.665924 90.95143\t540103200000\t西藏自治区;拉萨市;堆龙德庆区;羊达乡',
 '西藏第三建筑工程公司\t西藏自治区拉萨市城关区色拉路85号\t540102\t公司企业;公司\t29.686093 91.133712\t540102202000\t西藏自治区;拉萨市;城关区;娘热乡',
 '侠军牛肉拉面\t拉萨市城关区幸福新村113号客栈娘热中路\t540102\t美食;小吃快餐店\t29.674561 91.119735\t540102202000\t西藏自治区;拉萨市;城关区;娘热乡',
 '昌都市妇幼保健院\t西藏自治区昌都市卡若区通夏村\t540302\t医疗;专科医院\t31.154015 97.18847\t540302100000\t西藏自治区;昌都市;卡若区;城关镇',
 '西藏拉洁罗户外运动有限责任公司\t西藏自治区拉萨市城关区扎细街道夺底中路72号蜀园小区7栋2单元11号\t540102\t公司企业\t

In [127]:
get_baidu_content('西藏自治区拉萨市城关区嘎玛贡桑街道当热东路46号拉萨市农畜产品检测中心[1900]绿白', '13')['content']
#get_baidu_poi('西藏自治区拉萨市城关区曲米路警务站斜对面')

[{'acc_flag': 0,
  'addr': '西藏自治区拉萨市城关区',
  'addr_hot_percent': 0,
  'address_color_shows': [{'end': 22, 'name': '西藏自治区拉萨市城关区', 'start': 0}],
  'api_admin_info': {'area_code': 853,
   'area_name': '城关区',
   'city_code': 100,
   'city_name': '拉萨市',
   'prov_code': 13,
   'prov_name': '西藏自治区'},
  'area': 853,
  'area_name': '拉萨市城关区',
  'biz_type': 0,
  'brand_id': None,
  'catalogID': 0,
  'cla': [[4, '道路'], [43, '乡道']],
  'click_flag': 0,
  'cp': 'GuoKe',
  'detail': 0,
  'diPointX': 1014716331,
  'diPointY': 343844619,
  'di_tag': '道路',
  'dis': -1,
  'dist2route': 0,
  'dist2start': 0,
  'ext': {'detail_info': {'image': 'https://map-mobile-resource.cdn.bcebos.com/icon/rec/search/default/huanziditadoudi.png'}},
  'ext_display': None,
  'ext_type': 0,
  'f_flag': 0,
  'father_son': 0,
  'flag_type': '0',
  'geo': '1|10147163.31,3438446.19;10147163.31,3438446.19|10147163.31,3438446.19;',
  'geo_type': 0,
  'history_address': '拉萨市',
  'is_nearnest_nat': 0,
  'is_top_nat': 0,
  'name': '嘎玛

In [134]:
pd.DataFrame(poilist).to_csv('poi_xizang.txt', header=None, index=None)